In [1]:
import pandas as pd
from haystack.document_stores import OpenSearchDocumentStore
from haystack.schema import Document

In [2]:
ds = OpenSearchDocumentStore()

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.

In [49]:
with open('./data/processed_reviews.parquet', mode='rb') as file:
    df = pd.read_parquet(file)
    
with open('./industry_mapping.csv', mode='r') as file:
    df_industry = pd.read_csv(file)
    df_industry.rename(columns={i: f'industry_{i}' for i in df_industry.columns}, inplace=True)


In [52]:
df = pd.merge(df.loc[~df['profile_industry_id'].isna()], df_industry[['industry_id', 'industry_name']], how='inner', left_on='profile_industry_id', right_on='industry_id')

In [53]:
df.iloc[0].to_dict()

{'review_uuid': '56beb7dc-3973-486e-8924-7ec6fedd32c0',
 'company_id': 222766,
 'review_type': 'employee_review',
 'review_creation_date': Timestamp('2021-08-25 14:16:36+0000', tz='UTC'),
 'active': '1',
 'active_part': 'alles',
 'total_score': 4.3333,
 's_1': 4.0,
 's_2': 5.0,
 's_3': 5.0,
 's_4': 4.0,
 's_5': 4.0,
 's_6': 5.0,
 's_7': nan,
 's_8': 4.0,
 's_9': 4.0,
 's_10': 4.0,
 's_11': nan,
 's_12': nan,
 's_13': nan,
 's_14': nan,
 's_15': nan,
 's_16': nan,
 's_17': nan,
 's_18': nan,
 'job_status': 'REVIEW_STATUS_EX',
 'comment': None,
 'comment_1': 'Er hilft einem auch im privaten Bereich wenn es die Möglichkeit gibt.',
 'comment_2': None,
 'text_s1': 'ziele realistisch gesetzt teilweise übertroffen entscheidungen manchmal nicht nachvollziehbar irgendwie funtionieren etscheidungen trotzdem',
 'text_s2': 'kollegen halten gut zusammen macht echt spass',
 'text_s3': 'abwechslungsreich endergebniss passt darf arbeit möchte',
 'text_s4': 'firmenleitung legt wert positive arbeitsatmo

In [9]:
df.loc[df['text_s1_len'] > 10].groupby(by='company_name').size().sort_values(ascending=False)

company_name
FERCHAU GmbH                                 499
Netto Marken-Discount AG & Co. KG            439
ALDI SÜD                                     343
Hays                                         287
TEDi GmbH & Co. KG                           285
                                            ... 
Rudnick GmbH                                   1
GEUM.tec GmbH                                  1
Rudolf Dorsch                                  1
GETTO INNENARCHITEKTUR                         1
MWG - Wohnungsgenossenschaft eG Magdeburg      1
Length: 127780, dtype: int64

In [17]:
df.loc[df['company_name'] == 'FERCHAU GmbH', 'profile_uuid', 'job_status'].iloc[0]

'304de031-0ca6-45fe-b57e-99209d358551'

In [22]:
most_texts = '304de031-0ca6-45fe-b57e-99209d358551'

In [54]:
relevant_cols = [
    'text_s1', 
    'text_s1_original', 
    's_1',
    'profile_ebp_status',
    'profile_industry_id',
    'profile_weighted_total',
    'review_url',
    'industry_name']

In [55]:
df = df.loc[
    (df['profile_uuid'] == most_texts) & 
    (df['text_s1_len'] >= 20) &
    (df['text_s1_language'] == 'de') &
    (df['text_s1_language_confidence'] >= .75), 
    relevant_cols]

In [56]:
# orientstr {‘dict’, ‘list’, ‘series’, ‘split’, ‘tight’, ‘records’, ‘index’}
df.iloc[:2].to_dict(orient='index')

{814914: {'text_s1': 'eingehen zugehen mitarbeiter dankeschön gute laufende projekte bzw anerkennung gehör weiterbildungen dabei heisst slogan bringen',
  'text_s1_original': 'Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute laufende Projekte bzw. keine Anerkennung. Kein Gehör für Weiterbildungen. Dabei heißt es im Slogan "Wir bringen sie weiter". ',
  's_1': 1.0,
  'profile_ebp_status': 'EBP_PRO',
  'profile_industry_id': 38,
  'profile_weighted_total': 4.0774,
  'review_url': 'www.kununu.com/de/ferchau/bewertung/c152d20e-9d18-4088-ba95-9ef1a8ef61db',
  'industry_name': 'Dienstleistung'},
 815041: {'text_s1': 'inkompetentes verhalten führungsebene bspw kollegen bevorzugt gegeneinander ausgespielt schlecht gemacht schwächen stark hervorgehoben statt potenzial fördern umdenken nötig',
  'text_s1_original': 'Inkompetentes Verhalten der Führungsebene. Bspw. werden manche Kollegen bevorzugt, gegeneinander ausgespielt oder vor anderen schlecht gemacht. Schwächen werd

In [23]:
df.head()

,text_s1,text_s1_original,s_1,profile_ebp_status,profile_industry_id,profile_weighted_total
9632,eingehen zugehen mitarbeiter dankeschön gute laufende projekte bzw anerkennu...,Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute lau...,1.0,EBP_PRO,38,4.0774
11714,inkompetentes verhalten führungsebene bspw kollegen bevorzugt gegeneinander ...,Inkompetentes Verhalten der Führungsebene. Bspw. werden manche Kollegen bevo...,1.0,EBP_PRO,38,4.0774
20302,gros funktionen vorgesetzter ausübt quasi vorgesetzten beim kunden ausgeführ...,"Das Gros der Funktionen, die ein Vorgesetzter ausübt, wird durch den quasi-V...",5.0,EBP_PRO,38,4.0774
21547,normalerweise gut vorgesetzt zeit mal gewechselt eigentlicher vorgesetzt wur...,Normalerweise gut. Mein Vorgesetzt.er hat sich in der Zeit 3 mal gewechselt....,1.0,EBP_PRO,38,4.0774
24272,merkt vertrieblern tun porsche verkauft bekommt alten twingo ansonsten zulet...,"Man merkt, dass man es mit Vertrieblern zu tun hat. Einem wird ein Porsche v...",1.0,EBP_PRO,38,4.0774


In [77]:
def to_documents(
    data: pd.DataFrame, column_name: str = 's1') -> list[Document]:
    
    result = list()
    
    for index, values in data.to_dict(orient='index').items():
        d = Document(
            id=str(index),
            content=values[f'text_{column_name}_original'],
            meta={
                'stars': values[f's_{column_name[1:]}'], 
                'ebp': values['profile_ebp_status'], 
                'industry': values['industry_name'],
                'review_url': values['review_url'],
                'stars_profile': values['profile_weighted_total']})
        
        result.append(d)
        
        # todo: paragraph split
    
    return result

In [78]:
to_documents(df.head())[0]

<Document: {'content': 'Kein eingehen und zugehen auf die Mitarbeiter. Keine Dankeschön für gute laufende Projekte bzw. keine Anerkennung. Kein Gehör für Weiterbildungen. Dabei heißt es im Slogan "Wir bringen sie weiter". ', 'content_type': 'text', 'score': None, 'meta': {'stars': 1.0, 'ebp': 'EBP_PRO', 'industry': 38, 'stars_profile': 4.0774}, 'embedding': None, 'id': '9632'}>

In [40]:
ds.write_documents(to_documents(df))

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
from haystack.nodes import BM25Retriever


retriever = BM25Retriever(document_store=ds)

In [3]:
from haystack.nodes import FARMReader

# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader = FARMReader(model_name_or_path="deepset/gelectra-large-germanquad", use_gpu=True)


In [6]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [15]:
from pprint import pprint

# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="sind vorgesetzte authentisch?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})
#pprint(prediction)

/Users/paul.leitner/miniforge3/envs/text_preparation/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Inferencing Samples: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.66s/ Batches]


In [16]:
for i in prediction['answers'][:5]:
    print()
    pprint(i.to_dict())


{'answer': 'Vorgesetzte versuchen sich dem sog. Mainstream entsprechend zu '
           'Verhalten und sind hierdurch nicht authentisch',
 'context': 'Vorgesetzte versuchen sich dem sog. Mainstream entsprechend zu '
            'Verhalten und sind hierdurch nicht authentisch. Viele Vorgesetzte '
            'beim Kunden besitzen ',
 'document_id': '662395',
 'meta': {'ebp': 'EBP_PRO',
          'industry': 38,
          'stars': 3.0,
          'stars_profile': 4.0774},
 'offsets_in_context': [{'end': 109, 'start': 0}],
 'offsets_in_document': [{'end': 109, 'start': 0}],
 'score': 0.7375724911689758,
 'type': 'extractive'}

{'answer': 'Vorgesetzte gehen in Konfliktsituation mit der Situation souverän '
           'um, sind offen für alternativ Vorschläge der Mitarbeiter offen, '
           'hören sich diese an und beziehen die Mitarbeiter bis zu einem '
           'Bestimmten Grad mit in die Entscheidungen mit ein.',
 'context': 'Vorgesetzte gehen in Konfliktsituation mit der Situation